In [14]:
import pandasai as pai
from pandasai_openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

llm = OpenAI()  # reads OPENAI_API_KEY by default
pai.config.set({"llm": llm})

In [16]:
import pandas as pd
import pandasai as pai

# 1. Prepare your DataFrame however you like:
data = {
    "country": ["US", "UK", "FR", "DE", "IT"],
    "sales":   [5000, 3200, 2900, 4100, 2300]
}
df = pai.DataFrame(data)

# 2. Simply call `.chat()` with your prompt:
answer = df.chat("Which are the top 3 countries by sales?")

In [17]:
df.chat("what was my previous question?")

StringResponse(type='string', value='The country with the highest sales is US with sales of 5000.')

In [26]:
df.chat("Whcih country i should go to to study Data Science? and explain why with the data")

StringResponse(type='string', value='The best country to study Data Science is US because it has the highest sales figure of 5000, indicating a strong market for data science professionals and potentially better educational opportunities.')